In [23]:
%matplotlib inline

from functools import partial

import dask
from dask import delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cmlreaders import CMLReader, get_data_index
from cmlreaders.timeseries import TimeSeries

from thetamod import connectivity, tmi

In [2]:
subject = "R1286J"
experiment = "catFR3"
session = 0
rootdir = "~/mnt/rhino"

In [9]:
reader = CMLReader(subject, experiment, session, rootdir=rootdir)

In [10]:
events = reader.load("events")

In [20]:
stim_params = pd.DataFrame([row for row in events[events.type == "STIM_ON"].stim_params])

In [21]:
stim_params.head()

,amplitude,anode_label,anode_number,burst_freq,cathode_label,cathode_number,n_bursts,n_pulses,pulse_freq,pulse_width,stim_duration,stim_on
0,750.0,LA9,9,-1,LA10,10,-1,100,200,300,500,1
1,750.0,LA9,9,-1,LA10,10,-1,100,200,300,500,1
2,750.0,LA9,9,-1,LA10,10,-1,100,200,300,500,1
3,750.0,LA9,9,-1,LA10,10,-1,100,200,300,500,1
4,750.0,LA9,9,-1,LA10,10,-1,100,200,300,500,1


In [24]:
stim_pairs = np.unique([(row.anode_number - 1, row.cathode_number - 1) for _, row in stim_params.iterrows()])

In [25]:
stim_pairs

array([8, 9])

In [3]:
def get_resting_connectivity(subject, rootdir):
    df = get_data_index(rootdir=rootdir)
    sessions = df[(df.subject == subject) & (df.experiment == "FR1")].session
    
    # Read EEG data for "resting" events
    eeg_data = []
    for session in sessions:
        reader = CMLReader(subject, "FR1", session, rootdir=rootdir)
        
        rate = reader.load('sources')['sample_rate']
        events = connectivity.get_countdown_events(reader)
        resting = connectivity.countdown_to_resting(events, rate)
        eeg = connectivity.read_eeg_data(reader, resting, reref=True)
        eeg_data.append(eeg)
        
    eegs = TimeSeries.concatenate(eeg_data)
    conn = connectivity.get_resting_state_connectivity(eegs.to_mne())
    return conn

In [4]:
# get_resting_connectivity(subject, rootdir)

In [5]:
def build_tmi_pipeline(subject, experiment, session, rootdir="~/mnt/rhino"):
    """Constructs a dask pipeline to compute TMI.
    
    Parameters
    ----------
    subject
    experiment
    session
    rootdir
    
    """
    reader = CMLReader(subject, experiment, session, rootdir=rootdir)
    pairs = reader.load("pairs")
    
    get_eeg = partial(tmi.get_eeg,
                      subject=subject,
                      experiment=experiment,
                      session=session,
                      reref=True,
                      rootdir=rootdir)
    
    conn = delayed(get_resting_connectivity)(subject, rootdir)
    
    pre_eeg = delayed(get_eeg)("pre")
    post_eeg = delayed(get_eeg)("post")
    
    distmat = delayed(tmi.get_distances)(pairs)
    
    pre_psd = delayed(tmi.compute_psd)(pre_eeg)
    post_psd = delayed(tmi.compute_psd)(post_eeg)
    
    distance_regression = delayed(tmi.regress_distance)(pre_psd, post_psd, conn, distmat)
    
    result = delayed(tmi.compute_tmi)(distance_regression)
    
    return result

In [6]:
tmi_pipeline = build_tmi_pipeline(subject, experiment, session, rootdir)

In [7]:
# tmi_pipeline.visualize()

In [8]:
tmi_pipeline.compute(get=dask.get)

fmin=4.000 Hz corresponds to 4.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.250 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.


/Users/depalati/src/thetamod/thetamod/connectivity.py:144: RuntimeWarning: fmin=4.000 Hz corresponds to 4.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.250 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.
  verbose=False)


[0.         0.1009762  0.09814457 0.10323914 0.11522676 0.10314914
 0.08554031 0.08234351 0.06372746 0.08123913 0.08040939 0.0921874
 0.06597021 0.13070589 0.10241946 0.09096173 0.0615422  0.05781787
 0.02598551 0.03175357 0.02381232 0.03197924 0.04684961 0.03480446
 0.04877338 0.03945684 0.06217258 0.04750434 0.07097451 0.02949037
 0.03073432 0.06610858 0.04537884 0.04215947 0.0306056  0.05917793
 0.04001328 0.05494514 0.07045072 0.07505336 0.05403737 0.11017922
 0.04368431 0.06967767 0.13381255 0.10964307 0.09854312 0.06479228
 0.05701921 0.04810443 0.08121809 0.08295249 0.06941879 0.04695317
 0.10197243 0.09416056 0.04128329 0.04661798 0.04450091 0.04571926
 0.01518757 0.02710035 0.03285075 0.0296843  0.04184874 0.05213522
 0.11887674 0.05048631 0.07997588 0.09299529 0.05035206 0.03446802
 0.05641421 0.073236   0.07033294 0.04360354 0.06472431 0.07994915
 0.07376136 0.0752709  0.07154354 0.10108893 0.0265076  0.03968909
 0.04343229 0.04338563 0.02087308 0.04620096 0.04703631 0.04420

MissingDataError: exog contains inf or nans